In [5]:
import json
import os
# Assuming your original JSON is stored in a variable called original_json
def load_checkpoint():
    """Load competitions data from the checkpoint file if it exists."""
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'r') as f:
            try:
                data = json.load(f)
                print(f"Loaded checkpoint with {len(data)} entries.")
                return data
            except Exception as e:
                print("Error loading checkpoint:", e)
    return []

In [19]:
CHECKPOINT_FILE = '../data/competition_list.json'

In [20]:
json_data= load_checkpoint()

Loaded checkpoint with 552 entries.


In [22]:
# Add data_url to each competition
for competition in json_data:
    base_url = competition["competition_url"]
    competition["data_url"] = f"{base_url}/data"
    # rename slug to competition_name
    if "slug" in competition:
        competition["competition_name"] = competition.pop("slug")

In [25]:
# Save the modified JSON back to the file
with open(CHECKPOINT_FILE, 'w') as f:
    try:
        json.dump(json_data, f, indent=4)
        print(f"Saved modified data with {len(json_data)} entries.")
    except Exception as e:
        print("Error saving modified data:", e)

Saved modified data with 552 entries.


In [87]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random

In [26]:
url = "https://www.kaggle.com/competitions/titanic/overview"

In [ ]:
# Configure undetected_chromedriver options
options = uc.ChromeOptions()

# Enable headless mode if desired (note: some sites may behave differently in headless mode)
options.headless = True

# Disable automation flags that could hint at scraping
options.add_argument('--disable-blink-features=AutomationControlled')

# Optionally, set a randomized user-agent
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"
    " Chrome/92.0.4515.107 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko)"
    " Version/14.0.3 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko)"
    " Chrome/91.0.4472.114 Safari/537.36"
]
options.add_argument(f'--user-agent={random.choice(user_agents)}')

# Initialize undetected_chromedriver
driver = uc.Chrome(options=options)
driver.get(url)

# Use explicit wait for dynamic content to load
wait = WebDriverWait(driver, 15)
data = {}

# find a div with id = description
desc_header = wait.until(EC.presence_of_element_located((By.ID, "description")))
description_content = desc_header.text.strip()
data['Description'] = description_content

# find a div with id = description
eval_header = wait.until(EC.presence_of_element_located((By.ID, "evaluation")))
evaluation_content = eval_header.text.strip()
data['Evaluation'] = evaluation_content

# search a div with class = "sc-ipAaKu blEaCU"
sidebar_container = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sc-ipAaKu.blEaCU")))

def extract_fields(text):
    # Define the desired and ignored fields.
    desired_fields = {"Competition Host", "Prizes & Awards", "Participation", "Tags","Files", "Size", "Type"}
    ignore_fields = {"Table of Contents", "Description", "Evaluation", "Frequently Asked Questions", "Citation","License"}
    
    # Initialize the extracted data.
    extracted = {}
    for field in desired_fields:
        # For Participation, we want a nested dict; otherwise, use a list.
        if field == "Participation":
            extracted[field] = {}
        else:
            extracted[field] = []
    
    current_field = None
    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue  # skip empty lines
        
        # If the line is a header for a desired field, switch context.
        if line in desired_fields:
            current_field = line
            continue
        
        # If the line is an ignore field, stop capturing data.
        if line in ignore_fields:
            current_field = None
            continue
        
        if current_field:
            if current_field == "Participation":
                # Expected format: "1,363,292 Entrants" -> split into number and label.
                parts = line.split(" ", 1)
                if len(parts) == 2:
                    num_str, label = parts
                    try:
                        # Remove commas and convert to integer.
                        number = int(num_str.replace(",", ""))
                        extracted[current_field][label] = number
                    except ValueError:
                        # In case conversion fails, store the raw value.
                        extracted[current_field][label] = num_str
                else:
                    # If the format is unexpected, skip this line.
                    continue
            else:
                extracted[current_field].append(line)
    
    return extracted
# display the sidebar container
sidebar_content = extract_fields(sidebar_container.text)

data['Competition Host'] = sidebar_content.get("Competition Host", [])
data['Prizes & Awards'] = sidebar_content.get("Prizes & Awards", [])
data['Participants'] = sidebar_content.get('Participation', {}).get('Participants', 0)
data['Entrants'] = sidebar_content.get('Participation', {}).get('Entrants', 0)
data['Submissions'] = sidebar_content.get('Participation', {}).get('Submissions', 0)
data['Teams'] = sidebar_content.get('Participation', {}).get('Teams', 0)
data['Tags'] = sidebar_content.get("Tags", [])

# Optional: Random sleep before closing to mimic human behavior
time.sleep(random.uniform(2, 5))

driver.quit()

In [ ]:
def extract_fields(text,desired_fields = {"Competition Host", "Prizes & Awards", "Participation", "Tags"},ignore_fields = {"Table of Contents", "Description", "Evaluation", "Frequently Asked Questions", "Citation","License"}):
    # Initialize the extracted data.
    extracted = {}
    for field in desired_fields:
        # For Participation, we want a nested dict; otherwise, use a list.
        if field == "Participation":
            extracted[field] = {}
        else:
            extracted[field] = []
    
    current_field = None
    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue  # skip empty lines
        
        # If the line is a header for a desired field, switch context.
        if line in desired_fields:
            current_field = line
            continue
        
        # If the line is an ignore field, stop capturing data.
        if line in ignore_fields:
            current_field = None
            continue
        
        if current_field:
            if current_field == "Participation":
                # Expected format: "1,363,292 Entrants" -> split into number and label.
                parts = line.split(" ", 1)
                if len(parts) == 2:
                    num_str, label = parts
                    try:
                        # Remove commas and convert to integer.
                        number = int(num_str.replace(",", ""))
                        extracted[current_field][label] = number
                    except ValueError:
                        # In case conversion fails, store the raw value.
                        extracted[current_field][label] = num_str
                else:
                    # If the format is unexpected, skip this line.
                    continue
            else:
                extracted[current_field].append(line)
    
    return extracted

In [88]:
url = "https://www.kaggle.com/competitions/titanic/data"

In [89]:
# Configure undetected_chromedriver options
options = uc.ChromeOptions()

# Enable headless mode if desired (note: some sites may behave differently in headless mode)
options.headless = True

# Disable automation flags that could hint at scraping
options.add_argument('--disable-blink-features=AutomationControlled')

# Optionally, set a randomized user-agent
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"
    " Chrome/92.0.4515.107 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko)"
    " Version/14.0.3 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko)"
    " Chrome/91.0.4472.114 Safari/537.36"
]
options.add_argument(f'--user-agent={random.choice(user_agents)}')

# Initialize undetected_chromedriver
driver = uc.Chrome(options=options)
driver.get(url)

# Use explicit wait for dynamic content to load
wait = WebDriverWait(driver, 15)
# data = {}

# # find a div with id = description
# desc_header = wait.until(EC.presence_of_element_located((By.ID, "description")))
# description_content = desc_header.text.strip()
# data['Description'] = description_content

In [90]:
# find a div with id = description
desc_header = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sc-fqpjkJ.xUjLo")))
description_content = desc_header.text.strip()
data['Description'] = description_content

In [94]:
# find a div with id = description
sidebar_header = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sc-sgxPq.fEgtle")))
sidebar_content = sidebar_header.text.strip()

In [97]:
extract_fields(sidebar_content)

{'Files': ['3 files'],
 'Participation': {},
 'Competition Host': [],
 'Size': ['93.08 kB'],
 'Prizes & Awards': [],
 'Type': ['csv'],
 'Tags': []}